# Instacart Market Basket Analysis
### Which products will an Instacart consumer purchase again?

#### Data Introduction

The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order. The dataset is anonymized and contains a sample of over 3 million grocery orders from more than 200,000 Instacart users. For each user, we provide between 4 and 100 of their orders, with the sequence of products purchased in each order. We also provide the week and hour of day the order was placed, and a relative measure of time between orders. For more information, see the blog post accompanying its public release.
Blog post: https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2

In [1]:
import pandas as pd
import numpy as np

## Get Data

In [4]:
aisles = pd.read_csv('db/aisles.csv')
departments = pd.read_csv('db/departments.csv')
orders = pd.read_csv('db/orders.csv')
products = pd.read_csv('db/products.csv')
order_products_prior = pd.read_csv('db/order_products__prior.csv')

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [4]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [6]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [15]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [7]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [14]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [66]:
order_products_train = pd.read_csv('db/order_products__train.csv')
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [82]:
def unique_value_counts(df):
    for col in df.columns:
        print('Unique ', col, ': ', len(df[col].unique()))
        
def print_head(df):
    print(db.head())
    

def stats_accross_dbs(function_keyword):
    dbs = [aisles,departments,products,orders,order_products_prior]
    dbs_names = ['aisles','departments','products','orders','order_products_prior']

    for i in range(len(dbs)):
        print(dbs_names[i])
        db = dbs[i]
        
        if function_keyword == unique_value_counts:
            unique_value_counts(db)
       
        if function_keyword == print_head:
            print(db.head())
            
        if function_keyword == 'unique/len':
            unique_value_counts(db)
            print(len(db))
        
        #else: print('''function_keyword is not valid''')
        print('\n')
    

In [84]:
stats_accross_dbs('unique/len')

aisles
Unique  aisle_id :  134
Unique  aisle :  134
134


departments
Unique  department_id :  21
Unique  department :  21
21


products
Unique  product_id :  49688
Unique  product_name :  49688
Unique  aisle_id :  134
Unique  department_id :  21
49688


orders
Unique  order_id :  3421083
Unique  user_id :  206209
Unique  eval_set :  3
Unique  order_number :  100
Unique  order_dow :  7
Unique  order_hour_of_day :  24
Unique  days_since_prior_order :  32
3421083


order_products_prior
Unique  order_id :  3214874
Unique  product_id :  49677
Unique  add_to_cart_order :  145
Unique  reordered :  2
32434489




In [78]:
#stats_accross_dbs(print_head)

In [9]:
orders.columns

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')

In [67]:
orders.eval_set.unique()

array(['prior', 'train', 'test'], dtype=object)

In [68]:
orders.order_dow.unique()

array([2, 3, 4, 1, 5, 0, 6], dtype=int64)

In [74]:
orders_prior = orders[orders.eval_set == 'prior']
orders_train = orders[orders.eval_set == 'train']
orders_test = orders[orders.eval_set == 'test']

## Features
##### Approches
**<p>1. % chance for the product to be sold</p>**
* Top products of the day
* Top Products by time

**<p>2. % chance for customer to buy a product</p>**

##### Questions
1) all order_id in order_products_prior exist in orders order_id?
2) check add_to_cart_order ==> what is it? ==> matches up with number of items === YES 


In [115]:
op = order_products_prior.order_id
o = orders.order_id

print('count op', len(op))
print('count o', len(o)) 
print('difference: ', len(op)-len(o),'\n')

print('unique count op', len(op.unique()))
print('unique count o', len(o.unique())) 
print('difference: ', len(op.unique())-len(o.unique()))

count op 32434489
count o 3421083
difference:  29013406 

unique count op 3214874
unique count o 3421083
difference:  -206209


In [ ]:
opu = order_products_prior.order_id.unique()
ou = orders.order_id.unique()

count = 0
for i in opu:
    if i not in ou:
        count+=1
print(count)

In [113]:
len(order_products_prior.order_id.unique())

3214874